In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import train_test_split
#import tensorflow as tf
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, LeaveOneOut,StratifiedKFold

In [2]:
# Chargement du jeu de données
palu=pd.read_excel('C:\\Users\\ASUS\\Documents\\these\\code_these-master\\palu_impute001.xlsx')
# Chargement du jeu de données
palu1=pd.read_excel('C:\\Users\\ASUS\\Documents\\these\\code_these-master\\MORBIDITE.xls')


# Selection des features du paludisme
paluu=pd.DataFrame(palu, columns = [  'TEMPERATURE','S_M8_APPETIT', 'S_FATIGUE', 'S_ARTHRALGI',
                                    'S_T_DIGESTIF', 'S_VERTIGE', 'S_FRISSON', 'S_MYALGIE', 'S_DABDO', 
                                    'S_VOMISS', 'S_NAUSEE', 'S_CEPHALE', 'S_FIEVRE','Diagnostic'])


# Selection des features du paludisme
paluu1=pd.DataFrame(palu1, columns = [ 'TEMPERATURE', 'S_M8_APPETIT', 'S_FATIGUE', 'S_ARTHRALGI',
                                    'S_T_DIGESTIF', 'S_VERTIGE', 'S_FRISSON', 'S_MYALGIE', 'S_DABDO', 
                                    'S_VOMISS', 'S_NAUSEE', 'S_CEPHALE', 'S_FIEVRE','Diagnostic'])

# Nous séparons notre variable cible en deux parties. Une partie pour entrainer y_train et une partie pour tester y_test



# Nous séparons notre variable cible en deux parties. Une partie pour entrainer y_train et une partie pour tester y_test


palu2=paluu.append(paluu1)

# Selection des features du paludisme
X=palu2.iloc[:,0:13]       

#X=pd.DataFrame(palu2, columns = [  'TEMPERATURE', 'S_M8_APPETIT', 'S_FATIGUE', 'S_ARTHRALGI',
                                    #'S_T_DIGESTIF', 'S_VERTIGE', 'S_FRISSON', 'S_MYALGIE', 'S_DABDO', 
                                    #'S_VOMISS', 'S_NAUSEE', 'S_CEPHALE', 'S_FIEVRE'])
y = palu2['Diagnostic']
X1_train, MX_test, y1_train, My_test = train_test_split(X, y,test_size=0.2 ,random_state=0)

In [3]:
# Les differentes methodes pour faire une validation croisée(Cross Validation)
cv=KFold(5,random_state=0)
cv1= LeaveOneOut()
cv2=StratifiedKFold(5)  

In [4]:
from sklearn.tree import DecisionTreeClassifier
# Chercher les meilleurs paramètres du modele
param_grid={'max_depth': np.arange(12,20), 'criterion':['gini','entropy']}
grid=GridSearchCV( DecisionTreeClassifier(),param_grid, cv=cv2)
grid.fit(X1_train,y1_train)

#Afficher le meilleur score
print('best score',grid.best_score_)
# Afficher les meilleurs parametres
print('best score',grid.best_params_)
# Recuperer le meilleur modele fourni 
model1=grid.best_estimator_
print('best score',model1.score(MX_test,My_test))
pred1=model1.predict_proba(MX_test)


from sklearn.linear_model import LogisticRegression
rf= LogisticRegression()
from sklearn.linear_model import LogisticRegression
param_grid={'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter':[10, 100,500,1000, 2000, 3000,4000]}
grid2=GridSearchCV( LogisticRegression(),param_grid, cv=StratifiedKFold(5)  )
grid2.fit(X1_train,y1_train)
print('best score',grid2.best_score_)
print('best score',grid2.best_params_)
model2=grid2.best_estimator_
print('best score',model2.score(MX_test,My_test))
pred2=model2.predict_proba(MX_test)

from sklearn.naive_bayes import GaussianNB  
param_grid={'var_smoothing': np.linspace(0.1,1,10)}
grid3=GridSearchCV( GaussianNB(),param_grid, cv=StratifiedKFold(5)  )
grid3.fit(X1_train,y1_train)
print('best score',grid3.best_score_)
print('best score',grid3.best_params_)
model3=grid3.best_estimator_
print('best score',model3.score(MX_test,My_test))
pred3=model3.predict_proba(MX_test)

best score 0.807296570275723
best score {'criterion': 'gini', 'max_depth': 17}
best score 0.8086079354404841
best score 0.7953934095494285
best score {'max_iter': 500, 'solver': 'lbfgs'}
best score 0.795965030262273
best score 0.7978143913920646
best score {'var_smoothing': 0.1}
best score 0.7983860121049092


# Max Voting

The max voting method is generally used for classification problems. In this technique, multiple models are used to make 

predictions for each data point. The predictions by each model are considered as a ‘vote’. The predictions which we get

from the majority of the models are used as the final prediction.

In [10]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('dt', model1), ('lr', model2),('nb',model3)], voting='hard')
model.fit(X1_train,y1_train)
print(model.score(MX_test,My_test))

y_pred=model.predict(MX_test)
#print(model.score(y_pred,My_test))
print(confusion_matrix(My_test, y_pred))  
print(classification_report(My_test,y_pred))


0.8025554808338937
[[4773  725]
 [ 743 1194]]
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      5498
           1       0.62      0.62      0.62      1937

    accuracy                           0.80      7435
   macro avg       0.74      0.74      0.74      7435
weighted avg       0.80      0.80      0.80      7435



# Averaging

Similar to the max voting technique, multiple predictions are made for each data point in averaging. In 
this method, we take an average of predictions from all the models and use it to make the final prediction. 
Averaging can be used for making predictions in regression problems or while calculating probabilities for
classification problems.

For example, in the below case, the averaging method would take the average of all the values.

i.e. (5+4+5+4+4)/5 = 4.4

In [11]:
pred1=model1.predict(MX_test)
pred2=model2.predict(MX_test)
pred3=model3.predict(MX_test)

In [15]:

finalpred=(pred1+pred2+pred3)/3
finalpred

array([0.66666667, 0.33333333, 0.        , ..., 0.        , 1.        ,
       0.        ])

In [16]:
finalpred=pd.DataFrame(finalpred)

In [17]:
y_pred1=finalpred.iloc[:,0]

# Weighted Average

This is an extension of the averaging method. All models are assigned different weights defining the importance of each model 

for prediction. For instance, if two of your colleagues are critics, while others have no prior experience in this field, then 

the answers by these two friends are given more importance as compared to the other people.


# Stacking model

In [18]:
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((0
                        ,1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
    test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred


Now we’ll create two base models – decision tree and knn.

In [20]:
#model1 = DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1, train=X1_train,y=y1_train,test=MX_test,n_fold=2)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

#model2 = LogisticRegression()

test_pred2 ,train_pred2=Stacking(model=model2,train=X1_train,y=y1_train,test=MX_test,n_fold=2)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

#model3 = GaussianNB()

test_pred3 ,train_pred3=Stacking(model=model3,train=X1_train,y=y1_train,test=MX_test,n_fold=2)

train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)


Create a 4th model, logistic regression, on the predictions of the decision tree and knn models

In [21]:
df = pd.concat([train_pred1, train_pred2,train_pred3], axis=1)
df_test = pd.concat([test_pred1, test_pred2,test_pred3], axis=1)

In [23]:
model = LogisticRegression(random_state=1)
model.fit(df,y1_train)
model.score(df_test, My_test)

0.8026899798251513